In [15]:
# Import Libraries
import pandas as pd
import pickle as pk
import numpy as np
from collections import defaultdict
import time

In [16]:
# Import processed sample data from folder
infile = open("Stored Data/sample_review_w_aspects.pickle", "rb")
data_reviews = pk.load(infile)
infile.close()

data_reviews = data_reviews.dropna(axis="rows", how="any")
data_reviews.head()

,Hotel,City,Reviews,Date,Score,Aspect,Sentiment
0,Hotel Arena,"Amsterdam, Netherlands",Only the park outside of the hotel was beauti...,8/3/2017,2.9,"[(""beautiful"",""park"")]","{'park': [0.5994, 0]}"
1,Hotel Arena,"Amsterdam, Netherlands",No real complaints the hotel was great great ...,8/3/2017,7.5,"[(""real"",""complaints""), (""surroundings"",""compl...","{'offer': [0, 0], 'sandwich': [0, 0], 'complai..."
2,K K Hotel George,"London, United Kingdom",Very comfortable beds smart bathroom good sho...,8/3/2017,9.6,"[(""very comfortable"",""beds""), (""smart"",""bathro...","{'shower': [0.4404, 0], 'delight': [0, 0], 'ba..."
3,Apex Temple Court Hotel,"London, United Kingdom",Everything,8/3/2017,10.0,[],{}
4,Apex Temple Court Hotel,"London, United Kingdom",Great Hotel with wonderful staff and good cui...,8/3/2017,7.9,"[(""wonderful"",""staff""), (""good"",""cuisine"")]","{'cuisine': [0.4404, 0], 'staff': [0.5719, 0]}"


In [17]:
infile = open("Stored Data/sample_hotels.pickle", "rb")
data_hotels = pk.load(infile)
infile.close()

data_hotels.head()

,Hotel,Address,Score,Latitude,Longitude,City
0,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,52.360576,4.915968,"Amsterdam, Netherlands"
1,K K Hotel George,1 15 Templeton Place Earl s Court Kensington a...,8.5,51.491888,-0.194971,"London, United Kingdom"
2,Apex Temple Court Hotel,1 2 Serjeant s Inn Fleet Street City of London...,9.2,51.513734,-0.108751,"London, United Kingdom"
3,The Park Grand London Paddington,1 3 Queens Garden Westminster Borough London W...,7.7,51.514218,-0.180903,"London, United Kingdom"
4,Park Plaza County Hall London,1 Addington Street Lambeth London SE1 7RY Unit...,8.4,51.501400,-0.116009,"London, United Kingdom"


In [18]:
hotel_dict = {}

for row in data_hotels.itertuples():
    hotel_dict.setdefault("{}_{}".format(row[1],row[6]), [row[2],row[3],row[4],row[5],row[6]])

hotel_dict

{'Hotel des Tuileries Relais du Silence_Paris, France': ['10 rue Saint Hyacinthe 1st arr 75001 Paris France',
  8.0999999999999996,
  48.866170500000003,
  2.3315610000000002,
  'Paris, France'],
 'Hotel Parc Saint Severin Esprit de France_Paris, France': ['22 Rue de La Parcheminerie 5th arr 75005 Paris France',
  9.0999999999999996,
  48.851858399999998,
  2.3447798,
  'Paris, France'],
 'Royal Hotel Champs Elys es_Paris, France': ['33 avenue de Friedland 8th arr 75008 Paris France',
  7.9000000000000004,
  48.874046299999996,
  2.3001960000000001,
  'Paris, France'],
 'NH Wien Belvedere_Vienna, Austria': ['Rennweg 12a 03 Landstra e 1030 Vienna Austria',
  8.4000000000000004,
  48.195434800000001,
  16.383429,
  'Vienna, Austria'],
 'Hotel Grums Barcelona_Barcelona, Spain': ['Palaud ries 26 Sants Montju c 08004 Barcelona Spain',
  8.3000000000000007,
  41.3727844,
  2.1721133999999997,
  'Barcelona, Spain'],
 'My Bloomsbury_London, United Kingdom': ['11 13 Bayley Street Bedford Square

In [19]:
def get_city(hotel_dict):
    city_dict = {}
    
    for key, value in hotel_dict.items():
        if value[4] in city_dict:
            city_dict[value[4]] += 1
        else:
            city_dict.setdefault(value[4], 1)
        
    return city_dict

city_dict = get_city(hotel_dict)
city_dict

{'Amsterdam, Netherlands': 101,
 'Barcelona, Spain': 180,
 'London, United Kingdom': 360,
 'Milan, Italy': 116,
 'Paris, France': 274,
 'Vienna, Austria': 123}

In [20]:
def timeline(data_reviews):
    timeline_dict = {}
    aspect_dict = {}
    
    for row in data_reviews.itertuples():
        for key, value in row[7].items():
            if key in aspect_dict:
                aspect_dict[key] += 1
            else:
                aspect_dict.setdefault(key, 1)
            
            hotel_city = "{}_{}".format(row[1],row[2])
            
            if row[4] in timeline_dict:
                if hotel_dict[hotel_city][4] in timeline_dict[row[4]]:
                    if key in timeline_dict[row[4]][hotel_dict[hotel_city][4]]:
                        timeline_dict[row[4]][hotel_dict[hotel_city][4]][key].append([hotel_city, (value[0]+value[1])])
                    else:
                        timeline_dict[row[4]][hotel_dict[hotel_city][4]].setdefault(key, [[hotel_city, (value[0]+value[1])]])
                else:
                    timeline_dict[row[4]].setdefault(hotel_dict[hotel_city][4], {key: [[hotel_city, (value[0]+value[1])]]})
            else:
                timeline_dict.setdefault(row[4], {hotel_dict[hotel_city][4]: {key: [[hotel_city, (value[0]+value[1])]]} })
                
    return timeline_dict, aspect_dict
        
overall = timeline(data_reviews)
timeline_dict = overall[0]
aspect_dict = overall[1]

In [21]:
aspect_dict

{'hanging': 1,
 'ups': 1,
 'lounge': 41,
 'cyclist': 1,
 'rethink': 1,
 'pickpockets': 1,
 'exception': 2,
 'army': 1,
 'Bathrooms': 5,
 'SPA': 1,
 'mechanic': 2,
 'asap': 1,
 'Decor': 12,
 'Roof': 9,
 'foam': 2,
 'showerhead': 1,
 'Stains': 2,
 'toothbrushes': 1,
 '4PM': 1,
 'vet': 1,
 'toilet': 54,
 'host': 3,
 'toaster': 1,
 'drinkers': 1,
 'wardens': 1,
 'greed': 1,
 'dining': 22,
 'pay': 2,
 'Minories': 1,
 'page': 1,
 'mattresses': 7,
 'memory': 1,
 'potatoes': 1,
 'pickup': 1,
 'resturant': 1,
 'Unit': 1,
 'customers': 10,
 'seating': 12,
 'voyeur': 1,
 'material': 1,
 'coat': 2,
 'ratio': 1,
 'corridors': 15,
 'literature': 2,
 'diets': 1,
 'drinks': 72,
 'screech': 1,
 'fee': 4,
 'watts': 1,
 'times': 36,
 'tech': 9,
 'Fun': 1,
 'recruitment': 1,
 'preferences': 2,
 'themes': 1,
 'dents': 1,
 'driving': 1,
 'Wallpaper': 1,
 'glasses': 7,
 'douvet': 1,
 'cheese': 10,
 'Chose': 1,
 'pipes': 1,
 'chill': 4,
 'C10': 1,
 'restos': 1,
 'Railway': 1,
 'nesrside': 1,
 'balcon': 1,
 's

In [22]:
timeline_dict

{'7/29/2017': {'Amsterdam, Netherlands': {'11am': [['INK Hotel Amsterdam MGallery by Sofitel_Amsterdam, Netherlands',
     0]],
   'Amsterdam': [['WestCord Fashion Hotel Amsterdam_Amsterdam, Netherlands',
     0]],
   'Bed': [['Best Western Delphi Hotel_Amsterdam, Netherlands', 0.5106]],
   'Breakfast': [['citizenM Amsterdam_Amsterdam, Netherlands', 0.5719],
    ['INK Hotel Amsterdam MGallery by Sofitel_Amsterdam, Netherlands',
     -0.1027]],
   'Cleanliness': [['DoubleTree by Hilton Amsterdam Centraal Station_Amsterdam, Netherlands',
     0]],
   'Closing': [['art otel Amsterdam_Amsterdam, Netherlands', 0.4019]],
   'Dam': [['art otel Amsterdam_Amsterdam, Netherlands', 0]],
   'Front': [['Bilderberg Hotel Jan Luyken_Amsterdam, Netherlands', 0]],
   'Im': [['WestCord Fashion Hotel Amsterdam_Amsterdam, Netherlands',
     -0.4391]],
   'Kids': [['citizenM Amsterdam_Amsterdam, Netherlands', 0.5106]],
   'Nothing': [['Corendon Vitality Hotel Amsterdam_Amsterdam, Netherlands', 0],
    ['Ho

In [23]:
# Store the hotel data
outfile = open("Stored Data/hotel_dict.pickle", "wb")
pk.dump(hotel_dict, outfile)
outfile.close()

# Store the city data
outfile = open("Stored Data/city_dict.pickle", "wb")
pk.dump(city_dict, outfile)
outfile.close()

# Store the aspect data
outfile = open("Stored Data/aspect_dict.pickle", "wb")
pk.dump(aspect_dict, outfile)
outfile.close()

# Store the timeline data
outfile = open("Stored Data/timeline_dict.pickle", "wb")
pk.dump(timeline_dict, outfile)
outfile.close()